#MTRAIL creator script

In This assignment I'm going to create mtrails for the given graphs, which are represents networks. Mtrails are connected routes in the graphs and we can detect 1 bit error in the gven link with them.

Each mtrail is assigned to a link and all unique. The goals of this script are:

* reading the input file, which contains the graph's node number, the neighbour matrix and the incidency matrix.

* generate random  unique mtrails

* check if they are valid mtrails

* recreate few mtrails to make it valid

* and finally write it to the output file

In [0]:
from sys import version_info, stderr
if version_info <= (3,4,0):
    stderr.write("In order to run this script, you need Python 3.4.0 or above!")
    exit()

try:
    import numpy as np
except ImportError:
    print("You need to install numpy for this script!")
    exit()

from math import log, ceil
import collections

In [0]:
#declaring functions
def is_in_that(list_of_lists, test_list):
    """
    
    Checks if a row vector is in the matrix.
    INPUT:
        * list_of_lists: numpy 2 dimensional array
        * test_list: numpy 1 dimensional array
    OUTPUT:
        * list of indices. It shows which row(s) contains the row vector
        * if there is no instances of the row vector it returns a list which contaiuns -1
        
    list_of_list dimension is must be (MxN) and test_list dimension must be (1XN)
    It does not modify the input!
    """
    inds = [] # contains positive indices
    ind = 0
    
    for current_list in list_of_lists:
        # if the current row is equal to the parameter row vector
        if False not in (test_list == current_list):
          inds.append(ind)
        ind += 1
    
    if len(inds) == 0:
        return [-1]
    
    return inds
  

def generate_random_bits(b_amount, b_lenght):
    """
    
    Generates a random bit matrix.
    
    INPUT:
        * b_amount: number of rows
        * b_lenght: number of columns
    OUTPUT:
        * (b_amount x b_lenght) sized matrix of bits
        
    The output matrix does not contains a zero vector!
    It does not modify the input!
    
    """
    
    bits = np.random.randint(0, 2, size=(b_amount, b_lenght))    
    
    inds = is_in_that(bits, np.zeros(b_lenght)) # all zero vector(s) must be eliminated
    
    if inds[0] == -1:
        return bits
    else:
        for i in inds:
            bits[i] = np.bitwise_xor(bits[-1], bits[0])
        return bits
      
  
def where_is_not_unique(bits):
    """
    
    Shows which row vectors has a duplicate in the matrix.
    
    INPUT:
        * bits: matrix
    OUTPUT:
        * list of indices. It shows which row has multiple occurences.
        * [-1] if every row is unique
    
    """
    result = []
    
    unique_bits = np.unique(bits, axis=0)
    if bits.shape == unique_bits.shape:
        return [-1]
    else:
        for unique_bit_seq in unique_bits:
            result.append(is_in_that(bits, unique_bit_seq))
        return result
            
  
def set_unique(bits):
    """
    
    Create a bit matrix with unique rows.
    
    INPUT:
        * bits: bit matrix
    
    OUTPUT:
        * copy of bits with all unique rows.
    
    """
    are_all_unique = False
    copy_of_bits = np.copy(bits)
    
    while (not are_all_unique):
        result = where_is_not_unique(copy_of_bits)
        if result[0] == -1:
            are_all_unique = True
        else:
            for res in result:
                if len(res) > 1:
                    for i in range(0, len(res)-1):
                        copy_of_bits[res[i]] = generate_random_bits(1, bits.shape[1])
            
          
        if is_in_that(copy_of_bits, [0]*copy_of_bits.shape[1])[0] != -1:
            are_all_unique = False
            inds = is_in_that(copy_of_bits, [0]*copy_of_bits.shape[1])
            for ind in inds:
                copy_of_bits[ind] = generate_random_bits(1, copy_of_bits.shape[1])
                
    return copy_of_bits # all unique
  
def print_separator(sep_char="-"):
    """
    Decoration for printig on the console.
    
    INPUT:
        * sep_char: Character, which will occur 2x80 times
        
    OUTPUT:
        * on the screen 2x80 sep_char
    
    """
    s = ""
    for i in range(80):
        s += sep_char
    print("{}\n{}".format(s, s))
    
def number_of_mtrails(mtrails, inc_mat):
    number_of_mtrail = 0
    for i in range(len(mtrails[0])):
        M_=np.array([x[i] for x in mtrails])
        B_=inc_mat[:,np.where(M_==1)[0]]
        B_=np.array([x for x in B_ if (x != [0]*len(x)).any()])
        r=np.linalg.matrix_rank(B_)
        number_of_mtrail += len(B_) - r
        
    return number_of_mtrail
  
def get_links(nodes, inc_mat):
    """
    
    Create a dictionary about the edges. Contains the edge number from 0 to e-1, 
    where e is the number of the edges. And a list with 2 elements. It contains
    the nodes where the edge is.
    
    INPUT:
        * nodes: list of the nodes
        * inc_mat: incidency matrix
    
    OUTPUT:
        * dictionary with edges and the nodes of the edges
    
    It does not modify the input!
    
    """
    links = dict()
    
    for i in range(inc_mat.shape[1]):
        links[i] = []
    
    for edge in range(inc_mat.shape[1]):
        for node in range(nodes.shape[0]):            
            if (inc_mat[node][edge] != 0) and (links[edge] == []):
                links[edge].append(nodes[node])
                continue
            
            if (inc_mat[node][edge] != 0) and (len(links[edge]) < 2) and (len(links[edge]) >= 1):
                links[edge].append(nodes[node])
                break
    
    return links

First, I read the file. I created a list of file names. After chosing the file the code reads the number of nodes and matrices. The file contains coordinates about the nodes. For this script those coordinates are useless.

In [4]:
matrices_files = ["G8", "G52", "G202", "GER.txt", "EU", "IT"]
file_name = matrices_files[0]

print("Reading graph...")
print_separator()
f = open(file_name, "r")

size = int(f.readline())

print("Number of nodes = {}".format(size))

neigh_mat = np.zeros((size, size), dtype=int)
line = ""

# read the neighbour matrix
for i in range(size):
    line = list(f.readline().replace(" ", "").replace("\n", ""))

    for j in range(len(line)):
        neigh_mat[i][j] = int(line[j])
        
print("\nNeighbour matrix:\n{}".format(neigh_mat))


# to get the incidency matrix the data need a little transformation
lines =  []
for i in range(size):
    line = f.readline().replace(" ", "").replace("\n", "")
    line = line.replace("-1", "x")
    lines.append(line)

f.close()

print("\n\nNumber of edges = {}\n".format(len(lines[0])))
incidency_mat = np.zeros((size, len(lines[0])), dtype=int)
ind_n, ind_m = 0, 0

for line in lines:
    for k in range(0, len(line)):
        if line[k] == "x":
            incidency_mat[ind_n][ind_m] = -1
        else:
            incidency_mat[ind_n][ind_m] = int(line[k])
        ind_m += 1
    ind_n += 1
    ind_m = 0

print("Incidency matrix\n{}".format(incidency_mat))


Reading graph...
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Number of nodes = 8

Neighbour matrix:
[[0 1 1 0 0 0 0 0]
 [1 0 1 1 1 0 0 0]
 [1 1 0 0 1 0 0 0]
 [0 1 0 0 0 1 1 0]
 [0 1 1 0 0 1 0 0]
 [0 0 0 1 1 0 0 1]
 [0 0 0 1 0 0 0 1]
 [0 0 0 0 0 1 1 0]]


Number of edges = 11

Incidency matrix
[[ 1  1  0  0  0  0  0  0  0  0  0]
 [-1  0  1  1  1  0  0  0  0  0  0]
 [ 0 -1 -1  0  0  1  0  0  0  0  0]
 [ 0  0  0 -1  0  0  1  1  0  0  0]
 [ 0  0  0  0 -1 -1  0  0  1  0  0]
 [ 0  0  0  0  0  0 -1  0 -1  1  0]
 [ 0  0  0  0  0  0  0 -1  0  0  1]
 [ 0  0  0  0  0  0  0  0  0 -1 -1]]


After reading the graph data, the script calculates the number of mtrails and creating as many as many edge the graph has. It's all random, and it's more likely not correct. However every mtrail vector are unique!

In [5]:
print_separator()
print("Calculating the number of mtrails and creating them...")

n_edges = np.count_nonzero(incidency_mat == 1) # calculate it in a different way as well
bit_count = ceil(log(n_edges + 1, 2))
  
mtrails = generate_random_bits(n_edges, bit_count)
print("{}\n\n".format(mtrails))
sh = mtrails.shape

mtrails = set_unique(mtrails)
print(mtrails)

print("\nGenerated: {0}, Unique: {1}\n".format(sh, mtrails.shape))
print("For {0} edges we need {1} bits long mtrails therefore {1} mtrails.".format(n_edges, bit_count))

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Calculating the number of mtrails and creating them...
[[0 0 0 1]
 [0 1 0 1]
 [0 0 0 0]
 [1 1 0 0]
 [0 0 0 0]
 [1 0 1 0]
 [1 1 0 0]
 [1 1 1 0]
 [1 1 0 0]
 [1 1 0 1]
 [0 0 0 1]]


[[1 1 1 1]
 [0 1 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 1 0]
 [0 1 0 0]
 [1 1 1 0]
 [1 1 0 0]
 [1 1 0 1]
 [0 0 0 1]]

Generated: (11, 4), Unique: (11, 4)

For 11 edges we need 4 bits long mtrails therefore 4 mtrails.


In [6]:
number_of_mtrail = number_of_mtrails(mtrails, incidency_mat)

print("Current number of mtrails = {}".format(number_of_mtrail))
print("Optimal number of mtrails = {}".format(bit_count))

Current number of mtrails = 7
Optimal number of mtrails = 4


With the unique mtrails the script gives an mtrail to an edge.

Script creates a dictionary which has every edge with an index.

In [7]:
print_separator()
print("Allocating mtrails to the edges...")

nodes = np.arange(0, size, 1)
edges = get_links(nodes, incidency_mat)

print("\nEdges with random mtrails:")

for edge in range(n_edges):
    print("{}:   {}  -->  {}".format(edge, edges[edge], mtrails[edge]))


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Allocating mtrails to the edges...

Edges with random mtrails:
0:   [0, 1]  -->  [1 1 1 1]
1:   [0, 2]  -->  [0 1 0 1]
2:   [1, 2]  -->  [0 0 1 0]
3:   [1, 3]  -->  [1 0 0 0]
4:   [1, 4]  -->  [1 0 0 1]
5:   [2, 4]  -->  [1 0 1 0]
6:   [3, 5]  -->  [0 1 0 0]
7:   [3, 6]  -->  [1 1 1 0]
8:   [4, 5]  -->  [1 1 0 0]
9:   [5, 7]  -->  [1 1 0 1]
10:   [6, 7]  -->  [0 0 0 1]


In [35]:
def get_mtrail_edges(mtrails):
    mtrail_routes = dict()
    
    for bit_pos in range(mtrails.shape[1]):
        mtrail_routes[bit_pos] = []
        for mtrail_ind in range(mtrails.shape[0]):
            if mtrails[mtrail_ind][bit_pos] == 1:
                mtrail_routes[bit_pos].append(mtrail_ind)
    
    return mtrail_routes
  

test_route = np.array([[0,1,1], [1,0,1], [0,1,0]]) # it's an mtrail for a different graph
test_route1 = np.array([[0,1,1], [0,1,0]]) # mtrail
test_route2 = np.array([[0,0,1,0], [0,0,1,1], [0,1,1,1], [0,1,1,0]]) # not mtrail

routes = get_mtrail_edges(mtrails)
print("The random connected mtrails inices:\n{0}\n".format(routes))
print(edges)
print(mtrails)

The random connected mtrails inices:
{0: [0, 3, 4, 5, 7, 8, 9], 1: [0, 1, 6, 7, 8, 9], 2: [0, 2, 5, 7], 3: [0, 1, 4, 9, 10]}

{0: [0, 1], 1: [0, 2], 2: [1, 2], 3: [1, 3], 4: [1, 4], 5: [2, 4], 6: [3, 5], 7: [3, 6], 8: [4, 5], 9: [5, 7], 10: [6, 7]}
[[1 1 1 1]
 [0 1 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 1 0]
 [0 1 0 0]
 [1 1 1 0]
 [1 1 0 0]
 [1 1 0 1]
 [0 0 0 1]]


In [63]:
def make_connected(n_bits, edges, routes, mtrails):
    for mtrail in range(n_bits): # 0..4
        
        tmp_list = []
        tmp_set = set()
        node_cntr = []
        for mtrail_index in routes[mtrail]: # get connected trails indices
            pass
        print_separator()
    
"""
for i in routes[0]:
    print(edges[i])
print_separator()
for i in routes[1]:
    print(edges[i])
"""   
 
make_connected(bit_count, edges, routes, mtrails)

number_of_mtrails(mtrails, incidency_mat)

{0, 1, 2, 3, 4, 5, 6, 7}
[0, 1, 1, 3, 1, 4, 2, 4, 3, 6, 4, 5, 5, 7]
1: 0
3: 1
1: 2
3: 4
1: 6
1: 7
[]
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
{0, 1, 2, 3, 4, 5, 6, 7}
[0, 1, 0, 2, 3, 5, 3, 6, 4, 5, 5, 7]
1: 1
1: 2
1: 4
3: 5
1: 6
1: 7
[]
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
{0, 1, 2, 3, 4, 6}
[0, 1, 1, 2, 2, 4, 3, 6]
1: 0
1: 3
1: 4
1: 6
[]
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
{0, 1, 2, 4, 5, 6, 7}
[0, 1, 0, 2, 1, 4, 5, 7, 6, 7]
1: 2
1: 4
1: 5
1: 6
[]
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------


7

In [47]:
tmp_counter = np.array([], dtype=int)
print(tmp_counter)
#tmp_counter = np.delete(tmp_counter,0,0)
#print(tmp_counter)

[]


In [0]:
f = open("{0}_{1}.txt".format(file_name, "mtrails"), "w")

f.write("TALXCE\n")
for i in range(mtrails.shape[0]):
    for j in range(mtrails.shape[1]):
        f.write("{0} ".format(str(mtrails[i][j])))
    f.write("\n")

f.close()